In [10]:
import firebase_admin
from firebase_admin import firestore, credentials
from google.cloud.firestore_v1.base_query import FieldFilter, Or
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF

In [4]:
# Use a service account.
cred = credentials.Certificate('serviceAccountKey.json')

# Application Default credentials are automatically created.
firebase_admin.initialize_app(cred)     # Caso não funcione, inicializar o app dentro da variavel "app"
db = firestore.client()

# Note: Use of CollectionRef stream() is prefered to get()
docs = (
    db.collection("dbAusencias")
    .where(filter=FieldFilter("ocPeriodo", "==", "2405"))
    .stream()
)

In [5]:
# Criando uma lista para armazenar os dados
data = []

# Iterando sobre os documentos e adicionando os dados à lista
for doc in docs:
    data.append(doc.to_dict())

# Criando o DataFrame
df = pd.DataFrame(data)

In [6]:
display(df)

,ocHrFim,ocDtFim,funcUni,ocHrIni,zStamps,contrato,ocDtIni,funcEmp,ocPeriodo,zReadReceipt,zReadReceiptStamp,ocObs,funcNome,zValid,funcReg,ocTipo,docPic
0,,06/05/2024,Edifício Landscape,,[08/05/2024-07:19:52-Enviado-Robson-Teixeira-a...,20041754,06/05/2024,Focus Gestão e Facilities,2405,True,Abimael Nunes - 24/05/2024 16:25:05,-,Edson Aparecido Eleuterio de Jesus,True,218,Falta (Injustificada),https://firebasestorage.googleapis.com/v0/b/dm...
1,,12/05/2024,UPA - Campo dos Alemães,,[13/05/2024-12:23:08-Enviado-Maria do Socorro-...,20041754,12/05/2024,Facilitta Service,2405,True,Abimael Nunes - 27/05/2024 10:07:53,-,Silvana dos Santos da Silva,True,267,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...
2,,09/05/2024,Universidade Mandic,,[14/05/2024-11:56:28-Enviado-Bianca-de Andrade...,20041754,09/05/2024,Focus Gestão e Facilities,2405,True,Abimael Nunes - 24/05/2024 16:17:37,acompanhou a filha no hospital atestado da filha,Deidyane Aguiar,True,147,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...
3,,30/04/2024,,,[30/04/2024-21:01:52-Enviado-Maria-Amancio-Qk1...,20041754,30/04/2024,Focus Gestão e Facilities,2405,True,Abimael Nunes - 24/05/2024 15:45:44,segue o atestado da Tainá do dia de hoje,Taina Araujo,True,192,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...
4,,22/05/2024,Universidade Mandic,,[20/05/2024-14:58:15-Enviado-Bianca-de Andrade...,20041754,20/05/2024,Focus Gestão e Facilities,2405,True,Abimael Nunes - 24/05/2024 16:01:15,-,Maria Leuda,True,141,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,,11/05/2024,Hospital Pinhais,,[13/05/2024-11:53:27-Enviado-Roseli-Gonçalves-...,20041754,10/05/2024,Working,2405,True,Abimael Nunes - 24/05/2024 15:16:45,-,Amanda Mello,True,286,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...
100,,14/05/2024,Hospital Pinhais,,[14/05/2024-17:01:47-Enviado-Roseli-Gonçalves-...,20041754,14/05/2024,Working,2405,True,Abimael Nunes - 24/05/2024 15:14:20,-,Jezula Val,True,58,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...
101,,22/05/2024,Universidade Mandic,,[20/05/2024-14:59:36-Enviado-Bianca-de Andrade...,20041754,20/05/2024,Focus Gestão e Facilities,2405,True,Abimael Nunes - 24/05/2024 15:57:17,-,Adaise dos Santos,True,171,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...
102,,19/05/2024,,,[15/05/2024-09:30:09-Enviado-ALDENIRA MARIA -C...,20041754,13/05/2024,Facilitta Service,2405,True,Abimael Nunes - 27/05/2024 10:03:03,-,Rosimeire Francisquete Sousa,True,271,Falta (Atestada/Justificada),https://firebasestorage.googleapis.com/v0/b/dm...


In [11]:
# Convertendo 'ocPeriodo' para um formato mais legível (opcional)
df['ocPeriodo'] = pd.to_datetime(df['ocPeriodo'], format='%y%m').dt.strftime('%Y-%m')

# Agrupando e contando ocorrências por período
ocorrencias_por_periodo = df.groupby('ocPeriodo').size().reset_index(name='Total de Ocorrências')

# Agrupando e contando ocorrências por tipo
ocorrencias_por_tipo = df.groupby('ocTipo').size().reset_index(name='Total de Ocorrências')

# Agrupando e contando ocorrências por empresa
ocorrencias_por_empresa = df.groupby('funcEmp').size().reset_index(name='Total de Ocorrências')

# Agrupando e contando ocorrências por unidade
ocorrencias_por_unidade = df.groupby('funcUni').size().reset_index(name='Total de Ocorrências')


In [17]:
# Gráfico de barras por período
plt.figure(figsize=(10,10))
sns.barplot(x='Período', y='Total de Ocorrências', data=ocorrencias_por_periodo)
plt.title('Total de Ocorrências por Período')
plt.savefig('ocorrencias_por_periodo.png')
plt.close()

# Gráfico de pizza por tipo
plt.figure(figsize=(10,10))
plt.pie(ocorrencias_por_tipo['Total de Ocorrências'], labels=ocorrencias_por_tipo['ocTipo'], autopct='%1.1f%%')
plt.title('Total de Ocorrências por Tipo')
plt.savefig('ocorrencias_por_tipo.png')
plt.close()


In [21]:
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Adicionando o título
pdf.cell(200, 10, txt="Relatório de Ocorrências Operacionais", ln=1, align='C')
pdf.cell(200, 100, txt="", ln=1, align='C')

# Adicionando o gráfico de barras
pdf.image('ocorrencias_por_periodo.png', x=10, y=20, w=95)

# Adicionando o gráfico de pizza
pdf.image('ocorrencias_por_tipo.png', x=95, y=20, w=95)

# Adicionando a tabela de ocorrências por empresa
pdf.cell(100, 10, txt="Ocorrências por Empresa", ln=1, align='L')
for index, row in ocorrencias_por_empresa.iterrows():
    pdf.cell(50, 7, txt=str(row['funcEmp']), border=1)
    pdf.cell(50, 7, txt=str(row['Total de Ocorrências']), border=1, ln=1)

# Adicionando a tabela de ocorrências por unidade
pdf.cell(100, 10, txt="Ocorrências por Unidade", ln=1, align='L')
for index, row in ocorrencias_por_unidade.iterrows():
    pdf.cell(50, 7, txt=str(row['funcUni']), border=1)
    pdf.cell(50, 7, txt=str(row['Total de Ocorrências']), border=1, ln=1)

# Salvando o PDF
pdf.output("relatorio_ocorrencias.pdf")


''